# Importing modules and settings

Scanpy is based on pandas and numpy, and uses matplotlib to generate plots

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc

We import this library to quickly adjust plot sizes in an easy way

In [ ]:
from matplotlib.pyplot import rc_context

General settings of Scanpy

In [ ]:
sc.settings.verbosity = 3 
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')


# Declaring the input and output files

Scanpy saves the "analysis" in h5ad format. This is the output file and when you read it again you will have there all the analyses, markers, PCs, colors, etc, stored in there. We assign this to "results_file", and when we save data, we will just pass "results_file" to the writing data function.

In [ ]:
results_file = './schmidtea_asexual.h5ad'

Scanpy can read data in a variety of ways. The 10X format is one, and it is easy to obtain. This function expects the matrix in a directory with 2 tsv files and an mtx file. The data is written in an object called "adata". As usual, you can use the name you like here, you will just have to type the name in the arguments of each function you use on the object. "Adata" is the one the scanpy team uses in the tutorials.

In [ ]:
adata = sc.read_10x_mtx(
    "./matrix_format/",
    var_names='gene_symbols',
    cache=True)

This following cell will make the var names unique (by appending numbers). It should not be necessary, but it does not harm

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata.var_names

At this point let's check what we have in the 2 dataframes, adata.var and adata.obs, as well as the unstructured part, adata.uns

In [ ]:
adata.var

In [ ]:
adata.obs

In [ ]:
list(adata.var.columns)

In [ ]:
list(adata.obs.columns)

In [ ]:
list(adata.uns)

We can use this script to tell us what data do we have in a quick way

In [ ]:
print ("adata.obs has now", len(list(adata.obs.columns)), "columns:")
print(list(adata.obs.columns))
print ("adata.var has now", len(list(adata.var.columns)), "columns:")
print(list(adata.var.columns))
print ("adata.uns has now", len(list(adata.uns)), "items:")
print(list(adata.uns))

So, if you are running the analysis from scratch, there should not be much information in the pandas dataframes. The subsequent analyses will go adding layers of data

The default reporting of the anndata object does a similar thing. Just type "adata" (or the name of your anndata object) and it will print a summary

In [ ]:
adata

# Preprocessing

In the following cells several of the parameters of the analysis are set. We will chose the minimum counts, genes and cells, annotate mt genes if we know them, and produce some plots.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
sc.pp.filter_cells(adata, min_counts=150)
sc.pp.filter_cells(adata, min_genes= 50)
sc.pp.filter_genes(adata, min_cells=1)

**"n_counts" and "n_genes"** should be added now to **adata.obs**  
(the dataframe that contains the observations for each cell - therefore now we have a column that tells us how many UMI - counts and how many genes each cell expresses)  
**"n_cells"** should be added now to **adata.var**  
(the dataframe that contains the stats of each gene - we get a column that tells us in how many cells the gene is expressed)

In [ ]:
print ("adata.obs has now", len(list(adata.obs.columns)), "columns:")
print(list(adata.obs.columns))
print ("adata.var has now", len(list(adata.var.columns)), "columns:")
print(list(adata.var.columns))
print ("adata.uns has now", len(list(adata.uns)), "items:")
print(list(adata.uns))

In [ ]:
adata

These take the form of a pandas column (a pandas "series"). You can access them using pandas

In [ ]:
adata.obs['n_counts']

Using your pandas knowledge you can now check whether the function worked:

In [ ]:
print(adata.obs["n_counts"].min())
print(adata.obs["n_genes"].min())

...or check the maximum if you like, or anything else

In [ ]:
print(adata.obs["n_counts"].max())
print(adata.obs["n_genes"].max())

This following cell annotates and calculates the percentages of mitochondrial genes. In its current state it does nothing.

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, multi_panel=True, log = True)

In [ ]:
adata

# Matrix slicing and normalization

Let's first check the shape of our dataset in n_genes_by_counts (The number of genes with at least 1 count in a cell)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

Here we should slice the object to discard cells that are outside of the distribution - likely to be aggregates, doublets, etc

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 1000, :]

In [ ]:
adata = adata[adata.obs.total_counts < 5000, :]

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

The following 2 functions normalise and log transform the matrix

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

# Selecting highly variable genes

This is a key step in the analysis, since UMAP, clustering, etc, are run only on the **most variable genes** (those that contain cell type specificity information)

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes = 12000)

In [ ]:
sc.pl.highly_variable_genes(adata)

Scanpy saves the whole matrix as an object called adata.raw and keeps working with the matrix sliced to contain only highly variable genes

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata

Now "adata" has n_vars = highly variable genes 

The old .var dataframe with all genes can still be accessed like this

In [ ]:
adata.raw.var

# Scaling the data

In [ ]:
sc.pp.scale(adata, zero_center=False)

# Performing the PCA and kNN analysis

PCA analysis underlies most aspects of single cell analysis. The main parameter we have to chose here is the number of Principal Components used.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps = 200)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=200, log=True)

In [ ]:
adata

(the PCA analysis is stored in another part of the anndata object, and also in the adata.uns)

This builds a kNN (key Nearest Neighbors) graph. This one takes a while.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=75, n_pcs=50)

... and finally we compute the UMAP

In [ ]:
sc.tl.umap(adata, min_dist=0.3, spread = 1)

The UMAP has no clusters, it is just a 2D representation of the cells. We have not done clustering yet

In [ ]:
sc.pl.umap(adata)

But we can plot already the expression of genes in each cell (a "feature plot" in seurat). For example, this is Smedwi-1

In [ ]:
sc.pl.umap(adata, color=['SmMSTRG.11390'])

Below, different ways of obtaining the plots, with different options

In [ ]:
with rc_context({'figure.figsize': (10, 10)}):
    sc.pl.umap(adata, color=['SmMSTRG.11390'], size = 30, color_map = "viridis")

In [ ]:
with rc_context({'figure.figsize': (10, 10)}):
    sc.pl.umap(adata, color=['SmMSTRG.11390'], size = 30, color_map = "magma")

In [ ]:
with rc_context({'figure.figsize': (10, 10)}):
    sc.pl.umap(adata, color=['SmMSTRG.11390'], size = 30, color_map = "magma", frameon=False)

In [ ]:
sc.pl.umap(adata, color=['SmMSTRG.164', 'SmMSTRG.11390', 'SmMSTRG.17020', 'SmMSTRG.16348', 'SmMSTRG.8221', 'SmMSTRG.20313'])

In [ ]:
sc.pl.umap(adata,
           color=['SmMSTRG.19765', 'SmMSTRG.4403', 'SmMSTRG.19528', 'SmMSTRG.23477', 'SmMSTRG.201', 'SmMSTRG.684', 'SmMSTRG.6988'],
           color_map = 'Purples',
           size = 30,
           frameon=False, 
          )



In [ ]:
sc.pl.umap(adata,
           color=['SmMSTRG.19765', 'SmMSTRG.4403', 'SmMSTRG.19528', 'SmMSTRG.23477', 'SmMSTRG.201', 'SmMSTRG.684', 'SmMSTRG.6988'],
           color_map = 'viridis',
           size = 30,
           frameon=False, 
          )


In [ ]:
with rc_context({'figure.figsize': (10, 10)}):
    sc.pl.umap(adata, color=['SmMSTRG.2111'], size = 100, color_map = 'plasma')

Now we do the clustering, using the "leiden" algorithm. The main parameter is the resolution. This will be stored in adata.obs['leiden']

In [ ]:
sc.tl.leiden(adata, resolution = 2)

In [ ]:
adata

Note that the colors are not added by the leiden function, they are added by the umap function

In [ ]:
sc.pl.umap(adata, color=['leiden'])

In [ ]:
adata

In [ ]:
with rc_context({'figure.figsize': (15, 15)}):
    sc.pl.umap(adata, color='leiden', legend_loc='on data', title='', size = 50, frameon=False)

In [ ]:
adata

The colors are stored in adata.uns['leiden_colors']. This is just a python list of the n colours of the n clusters

In [ ]:
adata.uns['leiden_colors']

You could of course apply different clustering resolutions and save them all. Then you need to pass a key to store the new resolution as another pandas column using "key_added"

In [ ]:
sc.tl.leiden(adata, resolution = 1, key_added = "res 1 leiden")

In [ ]:
sc.pl.umap(adata, color=['res 1 leiden'])

In [ ]:
adata

In [ ]:
sc.pl.umap(adata, color=['leiden', 'res 1 leiden'],legend_loc='on data', legend_fontsize = 10)

In [ ]:
adata

# Marker gene analysis

Now we will perform the marker gene analysis using the rank_genes_groups function. 

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

The markers are stored in adata.uns, in a section called "rank_genes_groups", in the form of a dictionary.

In [ ]:
list(adata.uns)

In [ ]:
adata.uns['rank_genes_groups']

In [ ]:
list(adata.uns['rank_genes_groups'])

The easier way to operate with them is to put them in a pandas dataframe. The following command takes the first 10 markers, only the names, and makes a dataframe, under the name "markers_leiden"

In [ ]:
markers_leiden = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
markers_leiden

You could check the markers of the 2nd layer of clustering ('res 1 leiden')

In [ ]:
sc.tl.rank_genes_groups(adata, 'res 1 leiden', method='wilcoxon', key_added = "rank_genes_groups res 1 leiden")
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

In [ ]:
list(adata.uns)

In [ ]:
markers_leiden_res_1 = pd.DataFrame(adata.uns['rank_genes_groups res 1 leiden']['names']).head(10)

In [ ]:
markers_leiden_res_1

# Finally, save the data

You could also save the data earlier, but don't forget to save it at the end

In [ ]:
adata.write(results_file)